## CNN（卷积神经网络）来实现图像识别


In [2]:
import os
import glob
import cv2 
import math
import dlib
import datetime
import random 

from scipy.linalg import svd
from numpy import linalg as LA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical#Keras神经网络框架,与pytorch相似
from sklearn.model_selection import train_test_split
import tensorboard as tb 
import tensorflow as tf 



%matplotlib inline

#### 数据预处理

In [3]:
path = glob.glob("E:/face_dectection/figure/*/*.jpg")

In [4]:
len(path)

895

In [5]:
random.shuffle(path)
path[0:10]

['E:/face_dectection/figure\\person4\\page0_the_19_liuyifei.jpg',
 'E:/face_dectection/figure\\person0\\page1_the_41_zhaoliyin.jpg',
 'E:/face_dectection/figure\\person1\\person1page1_the_3_yangmi.jpg',
 'E:/face_dectection/figure\\person1\\person1page1_the_19_yangmi.jpg',
 'E:/face_dectection/figure\\person0\\page0_the_15_zhaoliyin.jpg',
 'E:/face_dectection/figure\\person1\\person1page1_the_44_yangmi.jpg',
 'E:/face_dectection/figure\\person3\\page1_the_34_dilireba.jpg',
 'E:/face_dectection/figure\\person4\\page2_the_59_liuyifei.jpg',
 'E:/face_dectection/figure\\person4\\page1_the_6_liuyifei.jpg',
 'E:/face_dectection/figure\\person2\\page1_the_46_liyitong.jpg']

In [6]:
images=[]
count=0
for img in path:
    if "jpg" in img:
        try:
            n = cv2.imread(img)#发送读取错误时跳过
            gray = cv2.cvtColor(n,cv2.COLOR_BGR2GRAY)
        except:
            images.append(np.zeros((50,50)))#填充错误值
            count+=1
            continue
        gray = cv2.resize(gray,(50,50))
        images.append(gray)
        count=count+1
print(count)

895


In [7]:
images= np.array(images)
len(images)

895

In [8]:
#建立标准
Y_map_num = np.zeros((len(images)))
i = 0
for img in path:
    if "jpg" in img:
        if "liuyifei" in img:
            Y_map_num[i]=0
        elif "yangmi" in img:
            Y_map_num[i]=1
        elif "dilireba" in img:
            Y_map_num[i]=2
        elif "liyitong" in img:
            Y_map_num[i]=3
        elif "zhaoliyin" in img:
            Y_map_num[i]=4
        i = i+1

In [9]:
print(len(Y_map_num))
Y_map_num[0:20]

895


array([0., 4., 1., 1., 4., 1., 2., 0., 0., 3., 4., 0., 2., 2., 0., 3., 0.,
       3., 3., 1.])

In [10]:
Y_map_label = [0 for x in range(len(images))]
i = 0
for img in path:
    if "jpg" in img:
        if "liuyifei" in img:
            Y_map_label[i]="liuyifei"
        elif "yangmi" in img:
            Y_map_label[i]="yangmi"
        elif "dilireba" in img :
            Y_map_label[i]="dilireba"
        elif "liyitong" in img:
            Y_map_label[i]="liyitong"
        elif "zhaoliyin" in img:
            Y_map_label[i]="zhaoliyin"
        i = i+1

In [11]:
print(len(Y_map_label))
Y_map_label[0:20]

895


['liuyifei',
 'zhaoliyin',
 'yangmi',
 'yangmi',
 'zhaoliyin',
 'yangmi',
 'dilireba',
 'liuyifei',
 'liuyifei',
 'liyitong',
 'zhaoliyin',
 'liuyifei',
 'dilireba',
 'dilireba',
 'liuyifei',
 'liyitong',
 'liuyifei',
 'liyitong',
 'liyitong',
 'yangmi']

to_categorical：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示。
其表现为将原有的类别向量转换为独热编码的形式。
onehot编码：https://blog.csdn.net/moyu123456789/article/details/83444140

In [12]:
Y_one_hot = to_categorical(Y_map_num)

In [13]:
# print(Y_one_hot)
# print(Y_map_num)
print(Y_one_hot.shape)
print(Y_map_num.shape)

(895, 5)
(895,)


In [14]:
help(np.reshape)

Help on function reshape in module numpy:

reshape(a, newshape, order='C')
    Gives a new shape to an array without changing its data.
    
    Parameters
    ----------
    a : array_like
        Array to be reshaped.
    newshape : int or tuple of ints
        The new shape should be compatible with the original shape. If
        an integer, then the result will be a 1-D array of that length.
        One shape dimension can be -1. In this case, the value is
        inferred from the length of the array and remaining dimensions.
    order : {'C', 'F', 'A'}, optional
        Read the elements of `a` using this index order, and place the
        elements into the reshaped array using this index order.  'C'
        means to read / write the elements using C-like index order,
        with the last axis index changing fastest, back to the first
        axis index changing slowest. 'F' means to read / write the
        elements using Fortran-like index order, with the first index
        c

In [15]:
images1 = images.astype('float32')
images1 = images / 255#进行归一化
images1 = images.reshape(-1,50,50, 1)

In [16]:
images1.shape

(895, 50, 50, 1)

In [17]:
images1[1]

array([[[223.],
        [224.],
        [229.],
        ...,
        [218.],
        [217.],
        [216.]],

       [[225.],
        [234.],
        [234.],
        ...,
        [221.],
        [220.],
        [218.]],

       [[235.],
        [234.],
        [235.],
        ...,
        [223.],
        [221.],
        [219.]],

       ...,

       [[233.],
        [236.],
        [238.],
        ...,
        [218.],
        [206.],
        [216.]],

       [[232.],
        [235.],
        [238.],
        ...,
        [217.],
        [196.],
        [208.]],

       [[236.],
        [236.],
        [240.],
        ...,
        [212.],
        [192.],
        [197.]]])

In [18]:
print(images.shape)

(895, 50, 50)


In [19]:
### 进行数据集的划分

In [20]:
# 划分测试集和训练集

In [21]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_Y,test_Y = train_test_split(images1, Y_one_hot, test_size=0.2, random_state=13)
train_X.shape,test_X.shape,train_Y.shape,test_Y.shape

((716, 50, 50, 1), (179, 50, 50, 1), (716, 5), (179, 5))

In [22]:
#划分验证集

In [23]:
train_X,valid_X,train_Y,valid_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=13)
train_X.shape,valid_X.shape,train_Y.shape,valid_Y.shape

((572, 50, 50, 1), (144, 50, 50, 1), (572, 5), (144, 5))

### keras使用文档
### https://keras.io/zh/

Sequential模型
Sequential模型字面上的翻译是顺序模型，给人的第一感觉是那种简单的线性模型，
但实际上Sequential模型可以构建非常复杂的神经网络，
包括全连接神经网络、卷积神经网络(CNN)、循环神经网络(RNN)、等等。
这里的Sequential更准确的应该理解为堆叠，通过堆叠许多层，构建出深度神经网络。

In [26]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU

In [27]:
###构建神经网络

In [28]:
batch_size = 128
epochs = 10
num_classes = 5

In [29]:
model = Sequential()
#添加二维卷积层
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='valid',input_shape=(50,50,1)))
#添加激活函数
model.add(LeakyReLU(alpha=0.1))
#添加对于空间数据的最大池化
model.add(MaxPooling2D((2, 2),padding='valid'))
#解决过拟合策略
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='linear',padding='valid'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='linear',padding='valid'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
model.add(Dropout(0.4))
#将输入展平。不影响批量大小
model.add(Flatten())
#全连接层
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))           
model.add(Dropout(0.3))
model.add(Dense(64, activation='linear'))

model.add(LeakyReLU(alpha=0.1))           
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

In [47]:
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
#view the model  from summay perspective

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 22, 22, 64)        0         
                                                        

In [33]:
###用神经网络来进行学习

In [34]:
print(train_X.shape)
print(train_Y.shape)
print(valid_X.shape)
print(valid_Y.shape)

(572, 50, 50, 1)
(572, 5)
(144, 50, 50, 1)
(144, 5)


In [50]:
#创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = r"D:\python\Face-detection-and-Emotion-recognition-system\%s" % stamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

In [37]:
help(tf.keras.callbacks.TensorBoard)

Help on class TensorBoard in module keras.callbacks:

class TensorBoard(Callback, keras.utils.version_utils.TensorBoardVersionSelector)
 |  TensorBoard(*args, **kwargs)
 |  
 |  Enable visualizations for TensorBoard.
 |  
 |  TensorBoard is a visualization tool provided with TensorFlow.
 |  
 |  This callback logs events for TensorBoard, including:
 |  
 |  * Metrics summary plots
 |  * Training graph visualization
 |  * Weight histograms
 |  * Sampled profiling
 |  
 |  When used in `Model.evaluate`, in addition to epoch summaries, there will be
 |  a summary that records evaluation metrics vs `Model.optimizer.iterations`
 |  written. The metric names will be prepended with `evaluation`, with
 |  `Model.optimizer.iterations` being the step in the visualized TensorBoard.
 |  
 |  If you have installed TensorFlow with pip, you should be able
 |  to launch TensorBoard from the command line:
 |  
 |  ```
 |  tensorboard --logdir=path_to_your_logs
 |  ```
 |  
 |  You can find more informa

In [51]:
model_train = model.fit(train_X, train_Y, 
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(valid_X, valid_Y),
                        callbacks=[tensorboard_callback])

Epoch 1/10
5/5 [==============================] - 2s 166ms/step - loss: 1.6932 - accuracy: 0.2692 - val_loss: 1.6830 - val_accuracy: 0.2847
Epoch 2/10
5/5 [==============================] - 1s 145ms/step - loss: 1.7281 - accuracy: 0.2657 - val_loss: 1.8826 - val_accuracy: 0.2639
Epoch 3/10
5/5 [==============================] - 1s 130ms/step - loss: 1.6918 - accuracy: 0.2517 - val_loss: 2.2425 - val_accuracy: 0.2778
Epoch 4/10
5/5 [==============================] - 1s 162ms/step - loss: 1.6336 - accuracy: 0.2972 - val_loss: 1.7572 - val_accuracy: 0.2708
Epoch 5/10
5/5 [==============================] - 1s 154ms/step - loss: 1.6265 - accuracy: 0.2657 - val_loss: 1.6839 - val_accuracy: 0.3194
Epoch 6/10
5/5 [==============================] - 1s 154ms/step - loss: 1.5986 - accuracy: 0.2867 - val_loss: 1.6654 - val_accuracy: 0.2778
Epoch 7/10
5/5 [==============================] - 1s 130ms/step - loss: 1.5872 - accuracy: 0.3007 - val_loss: 1.7690 - val_accuracy: 0.3194
Epoch 8/10
5/5 [====

In [45]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [52]:
%tensorboard --logdir logs 

Reusing TensorBoard on port 6006 (pid 9512), started 0:05:26 ago. (Use '!kill 9512' to kill it.)

In [ ]:
help(model.fit)

In [ ]:
model.save("CNN_model.h5")

In [ ]:
test_eval = model.evaluate(test_X, test_Y, verbose=1)

In [ ]:
## 进行模型评价

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
accuracy = model_train.history['accuracy']
val_accuracy = model_train.history['val_accuracy']
loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(len(accuracy))
#plot the loss and accuracy
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_types = model.predict(test_X)

In [ ]:
predicted_types.shape

In [ ]:
predicted_types = np.argmax(np.round(predicted_types),axis=1)

In [ ]:
test_Y = np.argmax(np.round(test_Y),axis=1)

In [ ]:
predicted_types.shape, test_Y.shape

In [ ]:
## view the correcr the result

In [ ]:
correct = np.where(predicted_types==test_Y)[0]

print ("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[50:59]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(50,50), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_types[correct], test_Y[correct]))
    plt.tight_layout()

In [ ]:
incorrect = np.where(predicted_types!=test_Y)[0]
print ("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(50,50), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_types[incorrect], test_Y[incorrect]))
    plt.tight_layout()

In [ ]:
## 进行模型的评价

In [ ]:
from sklearn.metrics import classification_report

target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(test_Y, predicted_types, target_names=target_names))